# DX 704 Week 8 Project

This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [42]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [43]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 1146


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [44]:
# YOUR CHANGES HERE


import random
import pandas as pd

simulator = RoverSimulator(16)

def random_policy(state, actions):
    return random.choice(actions)



Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [45]:
# YOUR CHANGES HERE
#given 

#for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
#    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)


#modified
rows = []
for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    rows.append({
        "curr_state":  curr_state,
        "curr_action": curr_action,
        "next_reward": next_reward,
        "next_state":  next_state
    })

df = pd.DataFrame(rows)
df.to_csv("log-random.tsv", sep="\t", index=False)
print(f"Saved log-random.tsv with {len(df)} rows.")
df.head()




Saved log-random.tsv with 32 rows.


,curr_state,curr_action,next_reward,next_state
0,1986,1,0,1995
1,1995,0,0,1875
2,1875,-1,0,1754
3,1754,0,0,1762
4,1762,1,0,1771


Submit "log-random.tsv" in Gradescope.

## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [46]:
# YOUR CHANGES HERE

#given

#for episode in range(32):
#    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
#        #print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)
#        pass

In [66]:
import pandas as pd
from collections import defaultdict

alpha = 1.0         
gamma = 0.9         
simulator = RoverSimulator(16)

Q = defaultdict(float)

rows = []

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        
        old_q = Q[(curr_state, curr_action)]
        
        next_actions = simulator.get_actions(next_state)
        max_next_q = max(Q[(next_state, a)] for a in next_actions)
        target = next_reward + gamma * max_next_q
        
        new_q = (1 - alpha) * old_q + alpha * target
        Q[(curr_state, curr_action)] = new_q

        rows.append({
            "curr_state": curr_state,
            "curr_action": curr_action,
            "next_reward": next_reward,
            "next_state": next_state,
            "old_value": old_q,
            "new_value": new_q
        })



Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

In [68]:
qlog = pd.DataFrame(rows)
qlog.to_csv("q-random.tsv", sep="\t", index=False)
print(f"Saved q-random.tsv with {len(qlog)} rows.")
qlog.head()


Saved q-random.tsv with 32768 rows.


,curr_state,curr_action,next_reward,next_state,old_value,new_value
0,3,1,0,12,0.0,0.0
1,12,1,0,13,0.0,0.0
2,13,1,0,6,0.0,0.0
3,6,-1,0,5,0.0,0.0
4,5,1,0,6,0.0,0.0


Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [69]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd
from collections import defaultdict

# Reset simulator and Q
simulator = RoverSimulator(16)
Q = defaultdict(float)

alpha = 1.0
gamma = 0.9
epsilon = 0.25
rng = np.random.default_rng(2025)

# hard-code epsilon=0.25. this is high but the environment is deterministic.
def epsilon_greedy_policy(state, actions):
    if rng.random() < epsilon:
        return int(rng.choice(actions))
    # Exploit best-known action (random tie-break)
    qs = np.array([Q[(state, a)] for a in actions], dtype=float)
    maxq = qs.max()
    best = [a for a, q in zip(actions, qs) if q == maxq]
    return int(rng.choice(best))

Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [70]:
# YOUR CHANGES HERE

rows = []
for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(
        epsilon_greedy_policy, max_steps=1024
    ):
        old_q = Q[(curr_state, curr_action)]

        next_actions = simulator.get_actions(next_state)
        max_next_q = max(Q[(next_state, a)] for a in next_actions)
        target = next_reward + gamma * max_next_q

        new_q = (1 - alpha) * old_q + alpha * target
        Q[(curr_state, curr_action)] = new_q

        rows.append({
            "curr_state":  curr_state,
            "curr_action": curr_action,
            "next_reward": next_reward,
            "next_state":  next_state,
            "old_value":   old_q,
            "new_value":   new_q,
        })

        if next_reward > 0:
            break

In [71]:
qlog = pd.DataFrame(rows)
qlog.to_csv("q-greedy.tsv", sep="\t", index=False)
print(f"Saved q-greedy.tsv with {len(qlog)} rows.")
qlog.head()

Saved q-greedy.tsv with 24401 rows.


,curr_state,curr_action,next_reward,next_state,old_value,new_value
0,2040,1,0,2041,0.0,0.0
1,2041,0,0,2041,0.0,0.0
2,2041,1,0,2042,0.0,0.0
3,2042,1,0,2043,0.0,0.0
4,2043,0,0,1915,0.0,0.0


Submit "q-greedy.tsv" in Gradescope.

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [ ]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
from collections import defaultdict


assert isinstance(simulator, RoverSimulator)
assert 'Q' in globals()

res = simulator.resolution
num_states = res * res * 8

def greedy_action(state):
    actions = simulator.get_actions(state)
    qvals = np.array([Q[(state, a)] for a in actions], dtype=float)
    maxq = qvals.max()

    best_actions = [a for a, q in zip(actions, qvals) if q == maxq]
    return min(best_actions)

Submit "policy-greedy.tsv" in Gradescope.

In [ ]:
rows = []
for s in range(num_states):
    a = greedy_action(s)
    rows.append({"state": s, "action": a})

policy_df = pd.DataFrame(rows)
policy_df.to_csv("policy-greedy.tsv", sep="\t", index=False)
print(f"Saved policy-greedy.tsv with {len(policy_df)} states.")
policy_df.head()

Saved policy-greedy.tsv with 2048 states.


,state,action
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1


## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

In [ ]:
import random
import numpy as np
import pandas as pd
from collections import defaultdict

assert isinstance(simulator, RoverSimulator)

GAMMA = 0.9
ALPHA = 1.0

EPOCHS = 60                 
MAX_STEPS = 256             
EPS_START = 0.35           
EPS_END   = 0.05            
EPS_DECAY = (EPS_END / EPS_START) ** (1.0 / max(EPOCHS - 1, 1))  
ACTIONS = [-1, 0, 1]

Q = defaultdict(float)

def greedy_action(state):
    qvals = [Q[(state, a)] for a in ACTIONS]
    maxq  = max(qvals)
    best  = [a for a, q in zip(ACTIONS, qvals) if q == maxq]
    return random.choice(best)

def epsilon_greedy(state, eps: float):
    if random.random() < eps:
        return random.choice(ACTIONS)
    return greedy_action(state)

def run_episode(start_state: int, eps: float, max_steps: int = MAX_STEPS):
    s = start_state
    for _ in range(max_steps):
        a = epsilon_greedy(s, eps)
        r, s_next = simulator.get_next_reward_state(s, a)

        best_next = max(Q[(s_next, ap)] for ap in ACTIONS)
        Q[(s, a)] = r + GAMMA * best_next

        if r > 0:
            break
        s = s_next

eps = EPS_START
init_states = list(simulator.initial_states)

for epoch in range(EPOCHS):
    random.shuffle(init_states)
    for s0 in init_states:
        run_episode(s0, eps, MAX_STEPS)
    eps *= EPS_DECAY  

res = simulator.resolution
num_states = res * res * 8


In [ ]:
rows = []
for s in range(num_states):

    qvals = np.array([Q[(s, a)] for a in ACTIONS], dtype=float)
    maxq = qvals.max()
    best_actions = [a for a, q in zip(ACTIONS, qvals) if q == maxq]
    a_star = min(best_actions)
    rows.append({"state": s, "action": a_star})

policy_opt = pd.DataFrame(rows)
policy_opt.to_csv("policy-optimal.tsv", sep="\t", index=False)
print(f"Saved policy-optimal.tsv with {len(policy_opt)} states (ε from {EPS_START} → {eps:.3f})")
policy_opt.head()

Saved policy-optimal.tsv with 2048 states (ε from 0.35 → 0.048)


,state,action
0,0,0
1,1,0
2,2,0
3,3,-1
4,4,-1


Submit "policy-optimal.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [ ]:
with open("acknowledgments.txt", "w") as f:
    f.write("""none
- Libraries used: none
- Worked alone 
- AI: None 
""")
print("Wrote acknowledgments.txt")

Wrote acknowledgments.txt
